# Тест на стабильность и регрессию - Save/Load Consistency

## Импорты

In [13]:
import numpy as np
import pandas as pd
import tempfile
import logging
import json
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

print(f'Библиотеки ипортированы:')
print('tensorflow', tf.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)

Библиотеки ипортированы:
tensorflow 2.19.0
numpy 2.1.3
pandas 2.3.0


## Настройки логирования

In [9]:
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger()

## Конфигурация

In [10]:
MODEL_PATH = "../model/model.keras"
TOKENIZER_PATH = "../model/tokenizer/tokenizer.json"
CSV_PATH = "../data/processed/dataset.csv"
MAX_LEN = 25

## Проверка конфигурации на наличие файлов

In [11]:
# Проверка наличия каждого файла
missing_files = []

if not os.path.isfile(MODEL_PATH):
    missing_files.append(MODEL_PATH)
if not os.path.isfile(TOKENIZER_PATH):
    missing_files.append(TOKENIZER_PATH)
if not os.path.isfile(CSV_PATH):
    missing_files.append(CSV_PATH)

# Вывод результата
if missing_files:
    print("Ошибка! Не найдены следующие файлы:")
    for file in missing_files:
        print(f"   - {file}")
else:
    print(f"Модель импортирована из пути: {MODEL_PATH}")
    print(f"Токенизатор импортирован из пути: {TOKENIZER_PATH}")
    print(f"Датасет импортирован из пути: {CSV_PATH}")
    print(f"Конфигурация гиперпараметра MAX_LEN установлена: {MAX_LEN}")

Модель импортирована из пути: ../model/model.keras
Токенизатор импортирован из пути: ../model/tokenizer/tokenizer.json
Датасет импортирован из пути: ../data/processed/dataset.csv
Конфигурация гиперпараметра MAX_LEN установлена: 25


## Загрузка модели, токенайзера и датасета

In [14]:
# Загрузка модели
# Проверка существования файла перед загрузкой
if os.path.isfile(MODEL_PATH):
    print(f"Файл модели найден: {MODEL_PATH}")
    model = load_model(MODEL_PATH)
    print("Модель успешно загружена.")
else:
    raise FileNotFoundError(f"айл модели не найден по пути: {MODEL_PATH}")

Файл модели найден: ../model/model.keras
Модель успешно загружена.


In [15]:
# Загрузка токенизатора
# Проверка существования файла перед загрузкой токенизатора
if os.path.isfile(TOKENIZER_PATH):
    print(f"Файл токенизатора найден: {TOKENIZER_PATH}")
    with open(TOKENIZER_PATH, "r", encoding="utf-8") as f:
        tokenizer = tokenizer_from_json(f.read())
    print("Токенизатор успешно загружен.")
else:
    raise FileNotFoundError(f"Файл токенизатора не найден по пути: {TOKENIZER_PATH}")

Файл токенизатора найден: ../model/tokenizer/tokenizer.json
Токенизатор успешно загружен.


In [16]:
# Загрузка датасета
# Проверка существования файла
if not os.path.isfile(CSV_PATH):
    raise FileNotFoundError(f"Файл не найден по пути: {CSV_PATH}")
print(f"Файл найден: {CSV_PATH}")

# Загрузка датасета
df = pd.read_csv(CSV_PATH)

# Проверка наличия колонки "noisy"
if "noisy" not in df.columns:
    raise ValueError("В датасете отсутствует колонка 'noisy'.")

# Получение первых 10 'грязных' слов
sample_texts = df["noisy"].astype(str).head(10).tolist()
print("Примеры 'грязных' слов:", sample_texts)

Файл найден: ../data/processed/dataset.csv
Примеры 'грязных' слов: ['Apenines', 'Appenines', 'Athenean', 'Atheneans', 'Bernouilli', 'Blitzkreig', 'Brasillian', 'Britian', 'Brittish', 'Ceasar']


## Подготовка данных

In [19]:
sample_texts = df["noisy"].astype(str).head(10).tolist()
logger.info(f"Примеры входных слов: {sample_texts}")

logger.info("Токенизация и паддинг входных данных...")
sequences = tokenizer.texts_to_sequences(sample_texts)
padded = pad_sequences(sequences, maxlen=MAX_LEN, padding="post")

Примеры входных слов: ['Apenines', 'Appenines', 'Athenean', 'Atheneans', 'Bernouilli', 'Blitzkreig', 'Brasillian', 'Britian', 'Brittish', 'Ceasar']
Токенизация и паддинг входных данных...


## Функция проверки

In [20]:
def check_save_load_consistency(model, sample_data, path, atol=1e-6):
    logger.info("Предсказания от исходной модели...")
    preds1 = model.predict(sample_data)

    logger.info(f"Сохраняем модель во временный файл: {path}")
    model.save(path)

    logger.info("Загружаем модель из файла...")
    loaded_model = keras.models.load_model(path)

    logger.info("Предсказания от загруженной модели...")
    preds2 = loaded_model.predict(sample_data)

    logger.info(f"Сравниваем предсказания (atol={atol})...")
    consistent = np.allclose(preds1, preds2, atol=atol)

    if consistent:
        logger.info("\n✅ Проверка пройдена: модель стабильно сохраняется.")
        logger.info("✅ Предсказания до и после одинаковые.")
    else:
        logger.warning("\n❌ Проверка провалена!")
        logger.warning("❌ Предсказания отличаются! Проверь структуру или веса.\n")

    return consistent

## Использование

In [21]:
logger.info("Запуск проверки стабильности модели...")
tmp_file = tempfile.NamedTemporaryFile(suffix=".keras", delete=False).name
ok = check_save_load_consistency(model, padded, tmp_file)

Запуск проверки стабильности модели...
Предсказания от исходной модели...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 685ms/step


Сохраняем модель во временный файл: C:\Users\Rreeo\AppData\Local\Temp\tmpuwwzzrpi.keras
Загружаем модель из файла...
Предсказания от загруженной модели...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step


Сравниваем предсказания (atol=1e-06)...

✅ Проверка пройдена: модель стабильно сохраняется.
✅ Предсказания до и после одинаковые.
